## Własna hierarchia wyjatków

In [6]:
import traceback

class ValidationError(Exception):
    def __init__(self, message, malformed_json=None):
        super().__init__(message)
        self.malformed_json = malformed_json

class AgeError(ValidationError): ...

class EmailError(ValidationError): ...

data_json = {
    "user1": {
        "name": "Adam",
        "age": 20,
        "email": "adam9876@wp.pl"
    },
    "user2": {
        "name": "Bartek",
        "age": 30,
        "email": "bartosz111interia.pl"
    },
    "user3": {
        "name": "Czesław",
        "age": 40.5,
        "email": "czesiek123@onet.pl" 
    }
}

def validate_age(age):
    if isinstance(age, int):
        return True
    else:
        raise AgeError(f"Age type must be an int, got instead: {type(age)},")

def validate_email(email):
    if "@" in email:
        return True
    else:
        raise EmailError(f"Invalid email: {email}")
    
def validate_user_data(user_data):
    try: 
        validate_age(user_data['age'])
        validate_email(user_data['email'])
    except (AgeError, EmailError) as e:
        e.malformed_json = user_data
        raise 

for user_number in data_json:
    user_data = data_json[user_number]
    try:
        validate_user_data(user_data)
    except ValidationError as e:
        print(e.malformed_json)
        # pass



{'name': 'Bartek', 'age': 30, 'email': 'bartosz111interia.pl'}
{'name': 'Czesław', 'age': 40.5, 'email': 'czesiek123@onet.pl'}


## Propagacja błędu

### 1.  Explicit chaining (raise...from), re-raise

In [2]:
import traceback

class CustomException(Exception): ...

def low_level():
    raise AttributeError("Attribute X not found!")

def mid_level():
    try:
        low_level()
    # explicit chaining
    except AttributeError as e:
        raise CustomException("Mid level cannot function properly without this attribute!") from e
    
def top_level():
    try:
        mid_level()
    # re-raise
    except CustomException as e:
        # logging error...
        print(f'Caught CustomError at the top level. Original cause: {e.__context__}')
        raise

if __name__ == '__main__':
    try:
        top_level()
    except Exception as e:
        print(e, e.__cause__)
        # traceback.print_exc()


Caught CustomError at the top level. Original cause: Attribute X not found!
Mid level cannot function properly without this attribute! Attribute X not found!


### 2. Implicit chaining, re-raising

In [4]:
import traceback

class CustomException(Exception): ...

def low_level():
    raise AttributeError("Attribute X not found!")

def mid_level():
    try:
        low_level()
    # implicit chaining
    except AttributeError as e:
        raise CustomException("Mid level cannot function properly without this attribute!")
    
def top_level():
    try:
        mid_level()
    # re-raise
    except CustomException as e:
        # logging error...
        print(f'Caught CustomError at the top level. Original cause: {e.__context__}')
        e.args()
        raise

if __name__ == '__main__':
    try:
        top_level()
    except Exception as e:
        print(e, e.__cause__)
        # traceback.print_exc()


Caught CustomError at the top level. Original cause: Attribute X not found!
'tuple' object is not callable None


### 3. Wrapping (suppressed chaining)

In [134]:
import traceback

class CustomException(Exception): ...

def low_level():
    raise AttributeError("Attribute X not found!")

def mid_level():
    try:
        low_level()
    # chaining
    except AttributeError as e:
        raise CustomException("Mid level cannot function properly without this attribute!") from None
    
def top_level():
    try:
        mid_level()
    # re-raise
    except CustomException as e:
        # logging error...
        print(f'Caught CustomError at the top level. Original cause: {e.__context__}')
        raise

if __name__ == '__main__':
    try:
        top_level()
    except Exception as e:
        print(e, e.__cause__)
        traceback.print_exc()


Caught CustomError at the top level. Original cause: Attribute X not found!
Mid level cannot function properly without this attribute! None


Traceback (most recent call last):
  File "/tmp/ipykernel_44390/2541271532.py", line 26, in <module>
    top_level()
  File "/tmp/ipykernel_44390/2541271532.py", line 17, in top_level
    mid_level()
  File "/tmp/ipykernel_44390/2541271532.py", line 13, in mid_level
    raise CustomException("Mid level cannot function properly without this attribute!") from None
CustomException: Mid level cannot function properly without this attribute!
